In [14]:
import torch 
from model_V2 import LLM, Config 
import onnxruntime as ort
from onnxruntime import SessionOptions

In [15]:
block_size = 512 # -> context length : char : 256, word : 64

emb_size = 384
head_nb = 6
block_nb = 6
vocab_size = 53 
LLM_config = Config(
        vocab_size = vocab_size,
        emb_size = emb_size,
        head_nb = head_nb,
        block_nb = block_nb,
        block_size = block_size
        )


#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
model = LLM(LLM_config)
model.load_state_dict(torch.load("checkpoints_ft/model.pt"))


<All keys matched successfully>

In [4]:
model.eval()
vocab_chars = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j','k','l','m','n','o','p','q','r','s','t','u','v','w','x', 'y', 'z', ' ', '!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', '[', '\n', ']', '{', '}', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])
stoi = {char: i for i, char in enumerate(sorted(vocab_chars))}
itos = {i: char for i, char in enumerate(sorted(vocab_chars))}


In [26]:
# Load the onnx model in a ort session
import onnxruntime as ort
from onnxruntime import SessionOptions

sess_options = SessionOptions()
sess = ort.InferenceSession("ONNX_saved/exported_model.onnx", sess_options=sess_options, providers=["CPUExecutionProvider"])

print(sess_options)

In [10]:
x = torch.Tensor([stoi[i] for i in 'begining\n']).type(torch.int32).unsqueeze(0).to(device)
block_size = 512 
with torch.no_grad():
    for i in range(100):
                x_cond = x if x.size(1) <= 512 else x[:, -512:]
           
                logits = model(x_cond)
                logits = logits[:, -1, :]
            
                probs = F.softmax(logits, dim=-1)
                next_tok = torch.multinomial(probs, num_samples=1)
                x = torch.cat([x, next_tok], dim=-1)
print(''.join([itos[i] for i in x.tolist()[0]]))

begining
{2} :  ine ! et's down service ?
{1} :  ome here at the office .
{2} :  ery much ! ould you tell me 


In [5]:
#Implement softmax and multinomial method
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)

def multinomial(probs, num_samples=1):
    samples = []
    for i in range(num_samples):
        x = np.random.uniform()
        for j, prob in enumerate(probs):
            x = x - prob
            if x <= 0:
                samples.append(j)
                break
    return samples

In [6]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [27]:
import numpy as np
import torch.nn.functional as F

x = []
for i in range(512-len('begining\n')):
    x.append(stoi[' '])
for i in 'begining\n' :
    x.append(stoi[i])
x_onnx = np.array([x]).astype(np.int32)
x_torch = torch.tensor([x]).type(torch.int32)
block_size = 512 
model.eval()
# print('x torch : ', x_torch)
# print('x onnx : ', x_onnx)
with torch.no_grad():


    full_txt = ''
    full_txt_gt = ''
   
    for i in range(100):

        if x_onnx.shape[1] > 512:
            x_onnx = x_onnx[:, -512:]

        if x_torch.shape[1] > 512:
            x_torch = x_torch[:, -512:]
     
        #print(x_onnx)
        output = sess.run(None, {"input": x_onnx.astype(np.int32)})[0]

        gt_output = model(x_torch)



        logits = output[:,-1,:]
        logits_gt = gt_output[:,-1,:]

        probs = softmax(logits)
        probs_gt = F.softmax(logits_gt, dim=-1)

        #print(probs)

        next_tok = np.array(multinomial(probs[0], num_samples=1))
        next_tok_gt = torch.multinomial(probs_gt, num_samples=1)

        x_torch = torch.cat([x_torch, next_tok_gt], dim=-1)
        #print('next tok : ', next_tok)
        x_onnx = np.concatenate([x_onnx, next_tok.reshape(1,1)], axis=1)

        full_txt_gt += itos[next_tok_gt.item()]

        full_txt += itos[(next_tok)[0]]
        print(full_txt)
        print(full_txt_gt)
        # full_txt_gt += itos[next_tok_gt.item()]


{
{
{2
{2
{2}
{2}
{2} 
{2} 
{2} :
{2} :
{2} : 
{2} : 
{2} :  
{2} :  
{2} :  h
{2} :  n
{2} :  h 
{2} :  no
{2} :  h ,
{2} :  no 
{2} :  h , 
{2} :  no l
{2} :  h ,  
{2} :  no li
{2} :  h ,  c
{2} :  no lic
{2} :  h ,  ca
{2} :  no lice
{2} :  h ,  can
{2} :  no lice 
{2} :  h ,  can 
{2} :  no lice .
{2} :  h ,  can  
{2} :  no lice . 
{2} :  h ,  can  t
{2} :  no lice .  
{2} :  h ,  can  t 
{2} :  no lice .   
{2} :  h ,  can  t d
{2} :  no lice .    
{2} :  h ,  can  t de
{2} :  no lice .     
{2} :  h ,  can  t dea
{2} :  no lice .      
{2} :  h ,  can  t deal
{2} :  no lice .       
{2} :  h ,  can  t deal 
{2} :  no lice .        
{2} :  h ,  can  t deal  
{2} :  no lice .         
{2} :  h ,  can  t deal   
{2} :  no lice .          
{2} :  h ,  can  t deal    
{2} :  no lice .           
{2} :  h ,  can  t deal     
{2} :  no lice .            
{2} :  h ,  can  t deal      
{2} :  no lice .             
{2} :  h ,  can  t deal       
{2} :  no lice .              
{2} :  h ,

KeyboardInterrupt: 